In [79]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.express as px

#pip install statsmodels
import statsmodels.api as sm

#pip install PyPortfolioOpt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting

In [80]:
#Download historical data for selected tickers using Yahoo Finance API
tickers = ['BTC-USD', 'TSLA', 'NVDA', 'ETH-USD', 'JPM', 'CZK=X']
data = yf.download(tickers, start = '2020-01-01', end = '2024-09-30')["Adj Close"].dropna()
data

[*********************100%%**********************]  6 of 6 completed


Ticker,BTC-USD,CZK=X,ETH-USD,JPM,NVDA,TSLA
Date,,,,,,
2020-01-02,6985.470215,22.633200,127.410179,122.852867,5.973122,28.684000
2020-01-03,7344.884277,22.657801,134.171707,121.231667,5.877517,29.534000
2020-01-06,7769.219238,22.704700,144.304153,121.135239,5.902165,30.102667
2020-01-07,8163.692383,22.531200,143.543991,119.075867,5.973619,31.270666
2020-01-08,8079.862793,22.614000,141.258133,120.004791,5.984823,32.809334
...,...,...,...,...,...,...
2024-09-23,63329.800781,22.438200,2648.546387,211.440002,116.260002,250.000000
2024-09-24,64301.968750,22.604000,2654.354980,211.589996,120.870003,254.270004
2024-09-25,63143.144531,22.389799,2579.388672,210.190002,123.510002,257.019989


In [81]:
#Pulling the last prices from each month
data_monthly = data.resample('M').last()
data_monthly

Ticker,BTC-USD,CZK=X,ETH-USD,JPM,NVDA,TSLA
Date,,,,,,
2020-01-31,9350.529297,22.882999,180.160172,115.991219,5.886478,43.371334
2020-02-29,8672.455078,23.002399,226.760498,101.750809,6.728047,44.532665
2020-03-31,6438.644531,24.736601,133.593567,78.896095,6.566867,34.933334
2020-04-30,8658.553711,24.898500,207.602051,84.789497,7.281349,52.125332
2020-05-31,9439.124023,24.344101,220.675125,86.161926,8.844340,55.666668
2020-06-30,9137.993164,23.795401,226.315002,83.284264,9.468728,71.987335
2020-07-31,11323.466797,22.066601,345.554657,86.402489,10.582314,95.384003
2020-08-31,11680.820312,21.876499,435.079742,89.576439,13.333631,166.106674
2020-09-30,10784.491211,23.129900,359.937866,86.071709,13.493192,143.003326


In [82]:
#Calculating daily returns
returns = np.log(data).diff().dropna()
returns

Ticker,BTC-USD,CZK=X,ETH-USD,JPM,NVDA,TSLA
Date,,,,,,
2020-01-03,0.050172,0.001086,0.051709,-0.013284,-0.016135,0.029203
2020-01-06,0.056166,0.002068,0.072803,-0.000796,0.004185,0.019072
2020-01-07,0.049527,-0.007671,-0.005282,-0.017147,0.012034,0.038067
2020-01-08,-0.010322,0.003668,-0.016053,0.007771,0.001874,0.048033
2020-01-09,-0.025165,0.003042,-0.016265,0.003645,0.010923,-0.022189
...,...,...,...,...,...,...
2024-09-23,0.002163,-0.001683,0.033585,0.001657,0.002239,0.048140
2024-09-24,0.015234,0.007362,0.002191,0.000709,0.038887,0.016936
2024-09-25,-0.018186,-0.009521,-0.028649,-0.006639,0.021607,0.010757


In [83]:
#Calculating monthly returns from the resampled monthly data
returns_monthly = np.log(data_monthly).diff().dropna()
returns_monthly

Ticker,BTC-USD,CZK=X,ETH-USD,JPM,NVDA,TSLA
Date,,,,,,
2020-02-29,-0.075281,0.005204,0.230048,-0.130988,0.133627,0.026424
2020-03-31,-0.297834,0.072685,-0.529092,-0.254395,-0.024248,-0.242781
2020-04-30,0.296230,0.006524,0.440821,0.072040,0.103279,0.400210
2020-05-31,0.086315,-0.022518,0.061069,0.016057,0.194462,0.065730
2020-06-30,-0.032422,-0.022797,0.025236,-0.033969,0.068217,0.257109
2020-07-31,0.214436,-0.075427,0.423223,0.036757,0.111189,0.281421
2020-08-31,0.031071,-0.008652,0.230379,0.036076,0.231105,0.554719
2020-09-30,-0.079839,0.055713,-0.189598,-0.039912,0.011896,-0.149762
2020-10-31,0.228021,0.012422,0.061633,0.027455,-0.076502,-0.100372


In [84]:
#Download data for the SP500 index
benchmark = yf.download('^GSPC', start = '2020-01-01', end = '2024-09-30')['Adj Close']
benchmark_returns = benchmark.pct_change().dropna()

[*********************100%%**********************]  1 of 1 completed


In [85]:
#Calculating monthly returns of the benchmark
benchmark_monthly = benchmark.resample('M').last()
benchmark_monthly_returns = np.log(benchmark_monthly).diff().dropna()
benchmark_monthly_returns

Date
2020-02-29   -0.087860
2020-03-31   -0.133668
2020-04-30    0.119421
2020-05-31    0.044287
2020-06-30    0.018221
2020-07-31    0.053637
2020-08-31    0.067719
2020-09-30   -0.040018
2020-10-31   -0.028056
2020-11-30    0.102146
2020-12-31    0.036449
2021-01-31   -0.011199
2021-02-28    0.025757
2021-03-31    0.041563
2021-04-30    0.051097
2021-05-31    0.005471
2021-06-30    0.021971
2021-07-31    0.022493
2021-08-31    0.028578
2021-09-30   -0.048738
2021-10-31    0.066858
2021-11-30   -0.008369
2021-12-31    0.042689
2022-01-31   -0.054018
2022-02-28   -0.031863
2022-03-31    0.035148
2022-04-30   -0.092068
2022-05-31    0.000053
2022-06-30   -0.087652
2022-07-31    0.087201
2022-08-31   -0.043367
2022-09-30   -0.098049
2022-10-31    0.076835
2022-11-30    0.052358
2022-12-31   -0.060782
2023-01-31    0.059921
2023-02-28   -0.026459
2023-03-31    0.034451
2023-04-30    0.014536
2023-05-31    0.002479
2023-06-30    0.062719
2023-07-31    0.030664
2023-08-31   -0.017875
2023-0

In [86]:
#Using the linear regression model to calculate the beta of a given stock
y = returns_monthly['NVDA'] #To calculate the beta, change the ticker here
X = benchmark_monthly_returns

X = sm.add_constant(X)

model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   NVDA   R-squared:                       0.394
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     35.05
Date:                Thu, 14 Nov 2024   Prob (F-statistic):           2.29e-07
Time:                        13:17:51   Log-Likelihood:                 43.467
No. Observations:                  56   AIC:                            -82.93
Df Residuals:                      54   BIC:                            -78.88
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0367      0.015      2.380      0.0

In [87]:
#Calculating the weights for the optimal portfolio
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# Optimize for maximal Sharpe ratio
ef_opt = EfficientFrontier(mu, S)
opt_weights = ef_opt.max_sharpe(risk_free_rate = 0.032)
cleaned_opt_weights = ef_opt.clean_weights()
print(cleaned_opt_weights)
ef_opt.portfolio_performance(verbose=True, risk_free_rate=0.032)

OrderedDict([('BTC-USD', 0.0), ('CZK=X', 0.0), ('ETH-USD', 0.21117), ('JPM', 0.0), ('NVDA', 0.78177), ('TSLA', 0.00706)])
Expected annual return: 89.2%
Annual volatility: 51.6%
Sharpe Ratio: 1.67


(0.8916047599413468, 0.5158964089236769, 1.666235207441653)

In [88]:
# Optimize for minimum volatility
ef_minVol = EfficientFrontier(mu, S)
minVol_weights = ef_minVol.min_volatility()
cleaned_minVol_weights = ef_minVol.clean_weights()
print(cleaned_minVol_weights)
ef_minVol.portfolio_performance(verbose=True)

OrderedDict([('BTC-USD', 0.01276), ('CZK=X', 0.90972), ('ETH-USD', 0.0), ('JPM', 0.06277), ('NVDA', 0.00799), ('TSLA', 0.00676)])
Expected annual return: 2.5%
Annual volatility: 9.9%
Sharpe Ratio: 0.05


(0.025136340288288667, 0.09926457104567071, 0.05174394282050022)

In [89]:
#Calculating the 10D VaR for the optimal portfolio
optimal_portfolio_returns = sum(cleaned_opt_weights[ticker] * returns[ticker] for ticker in tickers)
print(f"10D VaR for the optimal portfolio: {optimal_portfolio_returns.quantile(0.01) * np.sqrt(10):.2%}")

10D VaR for the optimal portfolio: -26.35%


In [90]:
#Calculating the 10D VaR for the minimum risk portfolio
minimal_volatility_portfolio_returns = sum(cleaned_minVol_weights[ticker] * returns[ticker] for ticker in tickers)
print(f"10D VaR for the minimum risk portfolio: {minimal_volatility_portfolio_returns.quantile(0.01) * np.sqrt(10):.2%}")

10D VaR for the minimum risk portfolio: -4.79%
